In [192]:
import matplotlib
import pathlib
import os.path
from os import path
import mne
import mne_bids #brain imaging data structure #convert dataset to bits compatible dataset, .fif is actual data, will be able to run tools that expect bits data
from bids_validator import BIDSValidator
import numpy as np

In [193]:
matplotlib.use('Qt5Agg')  #interactive

In [194]:
data_path="/Users/Asmita_Hajra/Downloads/bidsexport" #path of BIDS data

## Structure of directory

In [164]:
mne_bids.print_dir_tree(data_path)

|bidsexport/
|--- .DS_Store
|--- CHANGES
|--- README
|--- dataset_description.json
|--- epochs_epo.fif
|--- evokeds_ave.fif
|--- participants.json
|--- participants.tsv
|--- task-meditation_events.json
|--- code/
|------ run_mw_experiment6.m
|--- stimuli/
|------ cancel.wav
|------ depth.wav
|------ expe_over.wav
|------ mind_wandering.wav
|------ rate_meditation.wav
|------ rate_mw.wav
|------ rate_tired.wav
|------ resume.wav
|------ resumed.wav
|------ resumemed.wav
|------ self.wav
|------ starting.wav
|------ time.wav
|------ valence.wav
|--- sub-001/
|------ .DS_Store
|------ ses-01/
|--------- .DS_Store
|--------- eeg/
|------------ sub-001_ses-01_task-meditation_channels.tsv
|------------ sub-001_ses-01_task-meditation_eeg.bdf
|------------ sub-001_ses-01_task-meditation_eeg.json
|------------ sub-001_ses-01_task-meditation_events.tsv
|------------ .ipynb_checkpoints/
|--------------- sub-001_ses-01_task-meditation_channels-checkpoint.tsv
|--------------- sub-001_ses-01_task-me

## Report

In [165]:
print(mne_bids.make_report(data_path)) #path to bids directory #doubt, no authors thus error

KeyError: 'Authors'

## Reading BIDS data-> analyzing raw data



In [195]:
# currently for finding X of dataset
def load_and_epoch_data(n): # n= no. of subjects 
     # INPUT: NUMBER OF SUBJECTS DATASET TO BE LOADED
     # OUTPUT: list of 2d array (length = no. of subjects * 2)
    raw_data_np=[]
    raw_data=[]
    subject_id=''
    session_id=''
    for i in range(n): #0-23
        for j in range(3): # no. of sessions
            # i is the subject number
            # j is the session number
            if(i<9):
                subject_id="00"+str(i+1)
            else:
                subject_id="0"+str(i+1)
                
            session_id="0"+str(j+1)
            
            # check if path exists 
            # print("hello"+ str(i)+""+str(j))
            dataset_path_to_validate=data_path+"/sub-"+subject_id+"/ses-"+session_id
            if(path.isdir(dataset_path_to_validate)): #validating if these sessions exist
                list_of_files = os.listdir(dataset_path_to_validate+"/eeg") # Checking if the folder is empty
                if len(list_of_files) != 0:
                    bids_path = mne_bids.BIDSPath(subject=subject_id, 
                              session=session_id,
                              task='meditation',
                              # run='01', #doubt what is run number
                              datatype='eeg', #will have a folder with datatype as name
                              root=data_path)
#                 if(BIDSValidator().is_bids(dataset_path_to_validate+"/eeg")):
#                     print("hellp")
                    raw = mne_bids.read_raw_bids(bids_path)
                    # print(raw.shape)'RawEDF', object has no attribute 'shape'
                    data, times = raw[:] # converting to ndarray
                    print (data.shape)
                    raw_data_np.append(data)
                    raw_data.append(raw) # list of raw data

    return raw_data, raw_data_np

no_of_subjects= 24
raw_data, raw_data_np = load_and_epoch_data(no_of_subjects)

Extracting EDF parameters from /Users/Asmita_Hajra/Downloads/bidsexport/sub-001/ses-01/eeg/sub-001_ses-01_task-meditation_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /Users/Asmita_Hajra/Downloads/bidsexport/sub-001/ses-01/eeg/sub-001_ses-01_task-meditation_events.tsv.
The event "response" refers to multiple event values. Creating hierarchical event names.
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/4
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming event: response -> response/4
    Renaming event: response -> response/4
    Renaming event: response -> response/2
    Renaming event: response -> response/2
    Renaming 

In [180]:
len(raw_data_np)

40

In [181]:
type(raw_data_np)

list

In [ ]:
# Saving the ndarray
arr = np.empty(40, object) # 40 objects
arr[:] = raw_data_np
np.save('/Users/Asmita_Hajra/mne-research/stored_data.npy', arr) # save

In [80]:
# Loading the nd array
loaded_raw_data = np.load('/Users/Asmita_Hajra/mne-research/stored_data.npy', allow_pickle=True) # load


In [18]:
# finding maximum no. of events

for i in range(40):
    events=mne.find_events(raw_data[i]) 
    
#Observation-> Max no of events-> 2, 4, 8, 128, 254

<class 'mne.io.edf.edf.RawEDF'>
<class 'mne.io.edf.edf.RawEDF'>
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
87 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
87 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
101 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
107 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this ev


## About data
#### Stimulus is 128-> Onset of first Question
#### Response -> Response to question 1, 2 or 3
#### doubt what is STATUS-> 254
#### "2"-> Response 1 (this may be a response to question 1, 2 or 3)"
#### "4"-> Response 2 (this may be a response to question 1, 2 or 3)"
#### "8"-> Response 3 (this may be a response to question 1, 2 or 3)"
#### "16"-> involuntary response
#### "128"-> First question onset (most important marker)

## Finding info and visualizing for one subject's session

In [12]:
print(raw_data[3].info) # 68 EEG channels

<Info | 9 non-empty values
 bads: []
 ch_names: A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, A14, ...
 chs: 68 EEG, 10 MISC, 1 RESP, 1 STIM
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 50
 lowpass: 52.0 Hz
 meas_date: 2014-04-12 16:57:33 UTC
 nchan: 80
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>


In [170]:
print(type(raw_data[0]))
# raw_data[0].plot()
events = mne.find_events(raw_data[0]) # 3 events, [  2   4 128]

# raw_data[5].plot()
events = mne.find_events(raw_data[5]) # 4 events, [  2   4   8 128]

raw_data[21].plot()
events = mne.find_events(raw_data[21]) # 5 events, [  2   4   8 128 254]

#events are actually stored as annotations
raw_data[21].annotations[0] 
raw_data[21].annotations[1]
events, event_id = mne.events_from_annotations(raw_data[21])
print("events and event_id", events, event_id)
print("events array shape", events.shape)

# doubt difference in response/2 vs response/4 vs response/8
print("no of event response/8", len(events[events[:, 2] == 4])) #all rows with event no 8
print(raw_data[21].info)
print("measuring date", raw_data[21].info['meas_date'])
print("sampling frequency",raw_data[21].info['sfreq']) #number of samples/second-> 256
print("line frquency", raw_data[21].info['line_freq']) #line frequency
print("subject information", raw_data[21].info['subject_info'])
print("events are now annotations", raw_data[21].annotations)
print("bads", raw_data[21].info['bads']) #none here
print("10 channel names", raw_data[21].ch_names[:10])
print("info of a channel", raw_data[21].info['chs'][0])

<class 'mne.io.edf.edf.RawEDF'>
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
87 events found
Event IDs: [  2   4 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
79 events found
Event IDs: [  2   4   8 128]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
113 events found
Event IDs: [  2   4   8 128 254]
Used Annotations descriptions: ['STATUS', 'response/2', 'response/4', 'response/8', 'stimulus']
events and event_id [[  3162      0      1]
 [  8547      0      5]
 [  9803      0      2]
 [ 10769      0      4]
 [ 11444      0      3]
 [ 44471      0      5]
 [ 45711      0      3]
 [ 47385      0      4]
 [ 47970      0      3]
 [

## Asserting uniformity of data and applying montage

In [129]:
f=open("/Users/Asmita_Hajra/mne-research/biosemi64.txt","r")
lines=f.readlines()
result=[]
for x in lines:
    result.append(x.split('\t')[0])
f.close()

In [130]:
print(result)

['Site', 'Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'Nz', 'LPA', 'RPA']


In [196]:
biosemi_montage = mne.channels.make_standard_montage('biosemi64')
biosemi_montage.plot(show_names=True)

Creating RawArray with float64 data, n_channels=64, n_times=1
    Range : 0 ... 0 =      0.000 ...     0.000 secs
Ready.


<Figure size 1280x1280 with 1 Axes>

In [209]:
# making dictionary to map channels to 10-20 biosemi64 montage format

channel_keys = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'B31', 'B32']
montage_keys= ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'] #without Nz, LPA, RPA

res_channels = {channel_keys[i]: montage_keys[i] for i in range(len(channel_keys))}

print(res_channels)

# for i in range(40):
#     raw_data[i].rename_channels(res_channels)

{'A1': 'Fp1', 'A2': 'AF7', 'A3': 'AF3', 'A4': 'F1', 'A5': 'F3', 'A6': 'F5', 'A7': 'F7', 'A8': 'FT7', 'A9': 'FC5', 'A10': 'FC3', 'A11': 'FC1', 'A12': 'C1', 'A13': 'C3', 'A14': 'C5', 'A15': 'T7', 'A16': 'TP7', 'A17': 'CP5', 'A18': 'CP3', 'A19': 'CP1', 'A20': 'P1', 'A21': 'P3', 'A22': 'P5', 'A23': 'P7', 'A24': 'P9', 'A25': 'PO7', 'A26': 'PO3', 'A27': 'O1', 'A28': 'Iz', 'A29': 'Oz', 'A30': 'POz', 'A31': 'Pz', 'A32': 'CPz', 'B1': 'Fpz', 'B2': 'Fp2', 'B3': 'AF8', 'B4': 'AF4', 'B5': 'AFz', 'B6': 'Fz', 'B7': 'F2', 'B8': 'F4', 'B9': 'F6', 'B10': 'F8', 'B11': 'FT8', 'B12': 'FC6', 'B13': 'FC4', 'B14': 'FC2', 'B15': 'FCz', 'B16': 'Cz', 'B17': 'C2', 'B18': 'C4', 'B19': 'C6', 'B20': 'T8', 'B21': 'TP8', 'B22': 'CP6', 'B23': 'CP4', 'B24': 'CP2', 'B25': 'P2', 'B26': 'P4', 'B27': 'P6', 'B28': 'P8', 'B29': 'P10', 'B30': 'PO8', 'B31': 'PO4', 'B32': 'O2'}


In [22]:
# lets assert a few things to find distribution of data
# sfreq, no. of channels, whether they have valid subject info

# do with loaded raw data later
# print(type(loaded_raw_data)) 
# print(loaded_raw_data.shape) # 40
# print(loaded_raw_data[0].shape) # (80, 696576)
# for i in range(40): #0-23
#         assert(loaded_raw_data[i].shape[0]==80) # checking if all have equal channels

In [202]:
# lets assert a few things to find distribution of data
# sfreq, no. of channels, whether they have valid subject info

print(len(raw_data_np)) # 40
print(raw_data_np[0].shape) # (80, 696576)
for i in range(40): #0-23
        assert(raw_data_np[i].shape[0]==80) # checking if all have equal channels
        assert(raw_data[i].info['sfreq']==256) # checking if all have equal sfreq
        assert((raw_data[i].info['subject_info']['group']=='expert') or (raw_data[i].info['subject_info']['group']=='novice')) # checking if all have group defined
        assert((raw_data[i].info['highpass']==0) and (raw_data[i].info['lowpass']==52.0)) # all highpass and lowpass same        
        ############################
        ###Applying montage###
        raw_data[i].set_montage(biosemi_montage, on_missing="ignore") #missing-> ['GSR1', 'GSR2', 'Plet', 'Temp'].; DigMontage is subset
        ###Dropping non-eeg channels###
        raw_data[i].drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp'])
        #channel_indices=mne.pick_types(raw_data[i].info, eeg=True, exclude=['Temp', 'Plet', 'GSR1', 'GSR2'])
        channel_indices=mne.pick_types(raw_data[i].info, eeg=True)
        assert(len(channel_indices)==64) # all have 64 EEG channels
        print(raw_data[i].info['ch_names'])
        assert(len(raw_data[i].info['bads'])==0) # no bad channels


40
(80, 696576)
['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'Status']
['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'Status']
['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1'

In [218]:
for i in range(40): #0-23
            assert(raw_data[i].info['line_freq']==50) # power line frequency

In [212]:
#checking plotted sensors for one subject
raw_data[21].plot_sensors(ch_type='eeg')
raw_data[21].plot_sensors(kind='3d', ch_type='eeg')
# raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[]) #meg is included in eeg maybe;; exclude=[] means dont exclude bad
# len(raw_eeg.ch_names) selecting a subset of channels

['Fp1',
 'AF7',
 'AF3',
 'F1',
 'F3',
 'F5',
 'F7',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'C1',
 'C3',
 'C5',
 'T7',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'P1',
 'P3',
 'P5',
 'P7',
 'P9',
 'PO7',
 'PO3',
 'O1',
 'Iz',
 'Oz',
 'POz',
 'Pz',
 'CPz',
 'Fpz',
 'Fp2',
 'AF8',
 'AF4',
 'AFz',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT8',
 'FC6',
 'FC4',
 'FC2',
 'FCz',
 'Cz',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP8',
 'CP6',
 'CP4',
 'CP2',
 'P2',
 'P4',
 'P6',
 'P8',
 'P10',
 'PO8',
 'PO4',
 'O2',
 'Status']

### Filtering


In [216]:
raw_data[21].info['line_freq']

50

In [230]:
# A filter removes or attenuates parts of a signal. 
# Usually, filters act on specific frequency ranges of a signal — 
# for example, suppressing all frequency components above or below a certain cutoff value. 

# Artifacts that are restricted to a narrow frequency range can sometimes be repaired
# by filtering the data. Two examples of frequency-restricted artifacts
# are slow drifts and power line noise.

# doubt do slow drift artifact?

# Power line noise is an environmental artifact that manifests 
# as persistent oscillations centered around the AC power line 
# frequency. Power line artifacts are easiest to see on plots of 
# the spectrum, so we’ll use plot_psd() to illustrate. 
# We’ll also write a little function that adds arrows to the 
# spectrum plot to highlight the artifacts:

# Doubt
# Observation-> did not find power line noise
# Observation-> not sure about slow drift
raw_data[21].plot()



<MNEBrowseFigure size 2202x1054 with 4 Axes>

In [ ]:
# Diff frequency bands, alpha, beta, delta, theta
# The power of each frequency band tells something about mental state, doubt

# Delta-> deepest sleep, dreamless 0.5-4
# Alpha-> relaxed, yoga, reading (then alpha becomes higher than beta etc)
# Plot psd of electrodes, which elec do u notice the highest alpha/beta/delta

In [ ]:
# raw_eeg_cropped = raw_eeg.copy().crop(tmax=100) #accepts tmin and tmax #crop to durn of 100 milisec
# raw_eeg_cropped.times[-1] #last elem of .times attribute
# no need to crop the data
# raw_eeg_cropped_filtered = raw_eeg_cropped.filter(l_freq=0.1, h_freq=40)
# doubt learn more on filtering, band pass

In [232]:
# Plot PSD of the data

# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(2)

# raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
# raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)  #plot power spectral density

# ax[0].set_title('PSD before filtering')
# ax[1].set_title('PSD after filtering')
# ax[1].set_xlabel('Frequency (Hz)')
# fig.set_tight_layout(True)
# plt.show()

raw_data[21].plot_psd()  #plot power spectral density


Effective window size : 8.000 (s)


<MNELineFigure size 2000x700 with 2 Axes>

## Creating epochs and generating evoked responses (ERP/ERF)

In [238]:
# 2 create epoch we need-> where does epoch start rel to event onset, where does it end
# should we apply baseline correction

data_epochs=[]
for i in range(40): #0-23
    events, event_id = mne.events_from_annotations(raw_data[i])
    tmin = -1 # where starts in secs, 300 ms b4 event
    tmax = 1 # end of epoch after event onset
    # baseline = (None, 0) 
    # (st of bsline, end of baseline), bsline extends from none (beg of epoch ) to event onset
    # doubt baseline correction

    epochs = mne.Epochs(raw_data[i], #data to cut into epochs
                    events=events, #which event s to use np arr
                    event_id=event_id, #assign human readable names to epochs
                    tmin=tmin,
                    tmax=tmax,
#                     baseline=baseline,
#                     preload=True
                    preload=True) #data gets loaded into mem
    print(epochs)
    data_epochs.append(epochs)

Used Annotations descriptions: ['response/2', 'response/4', 'stimulus']
Not setting metadata
Not setting metadata
87 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 87 events and 513 original time points ...
0 bad epochs dropped
<Epochs |  87 events (all good), -1 - 1 sec, baseline -1 – 0 sec, ~22.2 MB, data loaded,
 'response/2': 41
 'response/4': 18
 'stimulus': 28>
Used Annotations descriptions: ['response/2', 'response/4', 'stimulus']
Not setting metadata
Not setting metadata
101 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 101 events and 513 original time points ...
0 bad epochs dropped
<Epochs |  101 events (all good), -1 - 1 sec, baseline -1 – 0 sec, ~25.8 MB, data loaded,
 'response/2': 44
 'response/4': 30
 'stimulus': 27>
Used Annotations descriptions: ['response

In [245]:
data_epochs[39].plot()
print(data_epochs[39].event_id)


{'response/2': 1, 'response/4': 2, 'response/8': 3, 'stimulus': 4}
Dropped 0 epochs: 
Channels marked as bad: none
